In [1]:
from copy import deepcopy

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import torch
import torch.optim as optim
import torch.nn.functional as F

In [2]:
# Define function to get maximum, minimum energy positions of molecule
def get_positions_from_mol(mol, e_min):
    # Generate conformers and calculate their energy
    AllChem.EmbedMultipleConfs(mol, 10)
    idx_energy_list = AllChem.UFFOptimizeMoleculeConfs(mol, maxIters=1000)

    # Get position of conformers with maximum, minimum energy
    energy_list = [el[1] for el in idx_energy_list]
    if e_min:
        energy = min(energy_list)   
        print("min Energy: ", energy)
    else:
        energy = max(energy_list)
        print("max Energy: ", energy)
    idx = energy_list.index(energy)
    conformer = mol.GetConformer(idx)
    pos = conformer.GetPositions()

    # Convert position values to torch tensors
    pos = torch.FloatTensor(pos)

    # Set gradient to max_pos variable (we will optimize max_pos to min_pos)
    if e_min:
        print("Target position:")
    else:
        pos = pos.float().requires_grad_(True)
        print("Initial position with gradient:")
    print(pos)
    return pos

In [3]:
# Define function for doing iteration
def optimize(pos, target, zero_grad=True, loss_type="l2", it=1000, lr=0.01):
    optimizer = optim.Adam([pos], lr=lr)
    for i in range(it):
        if zero_grad:
            optimizer.zero_grad()
        # L2 loss
        if loss_type == "l2":
            loss = F.mse_loss(pos.view(-1), target.view(-1))
        if loss_type == "l1":
            loss = torch.abs(pos-target).view(-1).sum(-1)
        loss.backward()
        optimizer.step()
        if loss < 0.01:
            print(f"Loss lower than 0.01 at {i}th iteration.")
            print(loss)
            print(pos)
            break
        if i == it-1:
            print(f"End of the {it} iteration.")
            print(loss)
            print(pos)

In [4]:
# Get molecule
smiles = "CCCC"
mol = Chem.MolFromSmiles(smiles)

### Without optimzer zero gradient
If we do not intialize the optimizer's gradient as zero, the gradient will be accumulated and it is hard to properly update the variables.

In [5]:
# Position with minimum energy
min_pos = get_positions_from_mol(mol, True)

min Energy:  -0.10454262826526749
Target position:
tensor([[-1.8928, -0.2013, -0.0800],
        [-0.5562,  0.5097, -0.0628],
        [ 0.5562, -0.5097,  0.0628],
        [ 1.8928,  0.2013,  0.0800]])


In [6]:
max_pos_no_opt = get_positions_from_mol(mol, False)
optimize(max_pos_no_opt, min_pos, zero_grad=False, loss_type="l2", it=1000)

max Energy:  1.0094345718047124
Initial position with gradient:
tensor([[ 1.5559, -0.5532, -0.1156],
        [ 0.6551,  0.5610,  0.3836],
        [-0.6601,  0.5608, -0.3753],
        [-1.5509, -0.5687,  0.1073]], requires_grad=True)
End of the 1000 iteration.
tensor(1.5046, grad_fn=<MseLossBackward>)
tensor([[ 0.2058, -1.5506,  1.3489],
        [-0.3643,  0.3310, -0.6021],
        [ 0.3042, -1.7829,  0.4125],
        [-0.2131, -1.1532, -1.1121]], requires_grad=True)


### With optimzer zero gradient

In [7]:
max_pos = get_positions_from_mol(mol, False)
optimize(max_pos, min_pos, zero_grad=True, loss_type="l2", it=1000)

max Energy:  1.009434572280784
Initial position with gradient:
tensor([[-1.5476, -0.5630,  0.1674],
        [-0.6718,  0.5615, -0.3527],
        [ 0.6707,  0.5603,  0.3566],
        [ 1.5487, -0.5588, -0.1713]], requires_grad=True)
Loss lower than 0.01 at 90th iteration.
tensor(0.0099, grad_fn=<MseLossBackward>)
tensor([[-1.8935, -0.1991, -0.0785],
        [-0.5560,  0.5100, -0.0655],
        [ 0.5559, -0.1859,  0.0654],
        [ 1.8934,  0.1013,  0.0783]], requires_grad=True)


### L2 Loss vs L1 Loss

L1 and L2 loss terms can be expressed as following equations.
$$L_1 = \sum^n_{i=1} |y_i - f\left( x_i\right)|$$
$$L_2 = \sum^n_{i=1} \left( y_i - f\left(x_i \right) \right)^2$$
The gradient of tensor is the derivative of loss terms with respect to each element in tensor. The derivative of L1 and L2 loss can be expressed as following equations.
$${\operatorname{d}\!L_1\over\operatorname{d}\!x} = \pm 1$$
$${\operatorname{d}\!L_2\over\operatorname{d}\!x} = -2f'\left(x\right)f\left(x\right)$$
Since the derivative of L1 loss only can have either +1 or -1, finding the local minimum with L1 loss is harder than L2 loss, which has continuous derivative value.

In [8]:
max_pos_l1 = get_positions_from_mol(mol, False)
optimize(max_pos_l1, min_pos, zero_grad=True, loss_type="l1", it=1000)

max Energy:  1.0094345710523354
Initial position with gradient:
tensor([[-1.5034,  0.6378,  0.2702],
        [-0.7215, -0.5396, -0.2815],
        [ 0.6719, -0.5805,  0.3203],
        [ 1.5531,  0.4823, -0.3090]], requires_grad=True)
Loss lower than 0.01 at 128th iteration.
tensor(0.0075, grad_fn=<SumBackward1>)
tensor([[-1.8919, -0.2018, -0.0792],
        [-0.5557,  0.5130, -0.0633],
        [ 0.5572, -0.5090,  0.0633],
        [ 1.8921,  0.2030,  0.0791]], requires_grad=True)


The **forward()** function in the torch computes the value of loss functions while **backward()** function computes the gradients of the learnable parameters with respect to loss.

The **Graph** in torch grad package is a copmutational graph which nodes are composed with mathematical operators such as sum or multiply, except for the case that user-defined variable. The **Leaf** in torch means the variables that every initial variables, not the result of mathematical operations. For example, suppose that there's a graph with following equations.

$$d_1 = w_1 \times x_1$$
$$d_2 = w_2 \times x_2$$
$$d_3 = d_1 + d_2$$
$$y = w_3 \times d_3$$
$$L = 10 - y$$

The nodes of the graph consists of $\times$, $+$, and $-$. Also, $x_1$, $x_2$, $w_1$, $w_2$, and $w_3$ are leaves of the graph that initialized by users. The following is the definition of leaf in torch documentation.
- All Tensors that have *requires_grad* which is **False** will be leaf Tensors by convention.
- For Tensors that have *requires_grad* which is **True**, they will be leaf Tensors if they were created by the user. This means that they are not the result of an operation and so **grad_fn** is None.
- Only leaf Tensors will have their *grad* populated during a call to *backward()*. To get *grad* populated for non-leaf Tensors, you can use *retain_grad()*.

Therefore, the definition in **backward()** in torch documentation: "Computes the gradient of current tensor w.r.t. graph leaves." means that torch automatically computes the derivatives of loss w.r.t. graph leaves via chain rule.

In [10]:
a = torch.rand(10, requires_grad=True)
a.is_leaf

True

In [11]:
# b is created by the addition operation
b = torch.rand(10, requires_grad=True) + 2
b.is_leaf

False

**torch.autograd.grad** act slightly different with **torch.autograd.backward**. The latter computes the sum of gradients of given tensors w.r.t. graph leaves, and the former computes and returns the sum of the gradients of outputs w.r.t. the inputs.

Both **grad()** and **backward()** has options of *retain_graph* and *create_graph*.

With *create_graph=True*, we are declaring that we want to do further operations on gradients, so that the autograd engine can create a backpropable graph for operations done on gradients. *retain_graph=True* declares that we will want to reuse the overall graph multiple times, so do not delete it after someone called **backward()**. During the **backward()**, torch goes backward multiple times of computation graph (sum, square, MSE, view, sum in below example). If we do not use *retain_graph=True*, then the gradients will be vanished after computing first derivative computation graph.

In [12]:
# helper function for generating grad objects
def generate_grad(retain_graph, create_graph):
    input = torch.ones(2, 2, requires_grad=True)
    pred = input + 2
    pred = pred ** 2
    target = torch.ones_like(input) * 9 - 0.2
    loss_fn = torch.nn.MSELoss()
    loss = loss_fn(pred, target)
    print(f"Loss: {loss:.3f}")
    
    gradient = torch.autograd.grad(outputs=loss, inputs=input, retain_graph=retain_graph, create_graph=create_graph)
    print(f"dloss/dinput:\n {gradient}")
    return input, gradient[0].view(-1).sum()

In [13]:
input, gradient = generate_grad(False, False)
print(f"gradient without retain graph and create graph: {gradient:.3f}")
try:
    gradient.backward()
    print(input) # Graph did not created therefore cannot do backward() operation
except RuntimeError as RE:
    print("#### EXCEPTION: ", RE)

Loss: 0.040
dloss/dinput:
 (tensor([[0.6000, 0.6000],
        [0.6000, 0.6000]]),)
gradient without retain graph and create graph: 2.400
#### EXCEPTION:  element 0 of tensors does not require grad and does not have a grad_fn


In [14]:
input_r, gradient_r = generate_grad(True, False)
print(f"gradient only with retain_graph: {gradient_r:.3f}")
try:
    gradient_r.backward()
    print(input_r) # Graph did not created therefore cannot do backward() operation
except RuntimeError as RE:
    print("#### EXCEPTION: ", RE)

Loss: 0.040
dloss/dinput:
 (tensor([[0.6000, 0.6000],
        [0.6000, 0.6000]]),)
gradient only with retain_graph: 2.400
#### EXCEPTION:  element 0 of tensors does not require grad and does not have a grad_fn


In [15]:
input_c, gradient_c = generate_grad(False, True)
print(f"gradient only with retain_graph: {gradient_c:.3f}")
try:
    gradient_c.backward(retain_graph=True)
    print(input_c) # Retain graph option crashed with create graph option, thus buffers are freed
except RuntimeError as RE:
    print("#### EXCEPTION: ", RE)

Loss: 0.040
dloss/dinput:
 (tensor([[0.6000, 0.6000],
        [0.6000, 0.6000]], grad_fn=<MulBackward0>),)
gradient only with retain_graph: 2.400
#### EXCEPTION:  Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.


In [16]:
input_rc, gradient_rc = generate_grad(True, True)
print(f"gradient only with retain_graph: {gradient_rc:.3f}")
try:
    gradient_rc.backward()
    print(input_rc) # Graph created and retained in backward() operation
except RuntimeError as RE:
    print("#### EXCEPTION: ", RE)

Loss: 0.040
dloss/dinput:
 (tensor([[0.6000, 0.6000],
        [0.6000, 0.6000]], grad_fn=<MulBackward0>),)
gradient only with retain_graph: 2.400
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


References:
- torch official documentation: https://pytorch.org/docs/stable/autograd.html
- https://blog.paperspace.com/pytorch-101-understanding-graphs-and-automatic-differentiation/
- https://stackoverflow.com/questions/46774641/what-does-the-parameter-retain-graph-mean-in-the-variables-backward-method
- Gradient, Jacobian, Hessian... : https://darkpgmr.tistory.com/132